In [61]:
# Importing necessary libraries
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

# Initialize lists to store the scraped data
product_names = []
prices = []
ratings = []
sellers = []

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}

In [62]:
# URL of the first page of search results
url = "https://www.amazon.in/s?rh=n%3A6612025031&fs=true&ref=lp_6612025031_sar"
# Send a GET Request to the url
response = requests.get(url)
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")
# Find all the product items on the page
product_items = soup.find_all('div', {'data-component-type': 's-search-result'})

# Iterate over each product item
for item in product_items:

    # Extract product name
    product_name = item.find('span', {'class': 'a-size-base-plus a-color-base a-text-normal'}).text.strip()
    product_names.append(product_name)

    # Extract rating
    rating = item.find('span', {'class': 'a-icon-alt'}).text.strip()
    ratings.append(rating)

    # Extract href and create complete link of the product
    href = item.find('a', {'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})
    link = 'https://www.amazon.in' + href['href']

    # Send a GET request to the product link
    response = requests.get(link)
    soup1 = BeautifulSoup(response.content, "html.parser")

    # Check if the product is out of stock and continue to next iteration if it is
    out_of_stock = soup1.find('div', {'id': 'outOfStock'})
    if out_of_stock is not None:
        continue

    # Extract price
    price = item.find('span', {'class': 'a-price-whole'}).text.strip()
    prices.append(price)

    # Extract seller name
    merchant_info = soup1.find('div', {'id': 'merchant-info'})
    seller_link = merchant_info.find('a')
    seller_name = seller_link.text.strip()
    sellers.append(seller_name)

In [63]:
# Create a DataFrame from the scraped data
data = {
    'Product Name': product_names,
    'Price': prices,
    'Rating': ratings,
    'Seller Name': sellers
}
df = pd.DataFrame(data)

# Save the dataframe to a csv file
df.to_csv('amazon_products_first_page.csv', index=True)

In [64]:
data

{'Product Name': ['MI Power Bank 3i 20000mAh Lithium Polymer 18W Fast Power Delivery Charging | Input- Type C | Micro USB| Triple Output | Sandstone Black',
  'MI 10000mAh Lithium Ion, Lithium Polymer Power Bank Pocket Pro with 22.5 Watt Fast Charging, Dual Input Ports(Micro-USB and Type C), Triple Output Ports, (Black)',
  'Mi 10000mAH Li-Polymer, Micro-USB and Type C Input Port, Power Bank 3i with 18W Fast Charging (Midnight Black)',
  'Ambrane 10000mAh Slim Power Bank, 20W Fast Charging, Dual Output, Type C PD (Input & Output), Quick Charge, Li-Polymer, Multi-Layer Protection for iPhone, Anrdoid & Other Devices (Stylo 10K, Black)',
  'Ambrane 20000mAh Power Bank, 20W Fast Charging, Tripe Output, Type C PD (Input & Output), Quick Charge, Li-Polymer, Multi-Layer Protection for iPhone, Smartphones & Other Devices (Stylo 20K, Blue)',
  'Ambrane 20000mAh Power Bank with 20W Fast Charging, Triple Output, Power Delivery, Type C Input, Made in India, Multi-Layer Protection, Li-Polymer + Typ